In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from multiprocessing import Pool

import pandas as pd
import os
import time
import re
import requests

PROXY = '139.5.135.37:5678'
options = webdriver.FirefoxOptions()
options.add_argument('--proxy-server=socks4://' + PROXY)
options.add_argument("--disable-notifications")
#options.add_argument("--headless")

SECS = 5
PATH = os.path.abspath(os.curdir) + '/geckodriver'

In [2]:
def get_html_page(url):
    try:
        driver = webdriver.Firefox(executable_path = PATH, options = options)
        driver.get(url=url)
        time.sleep(SECS)    
        html_page = driver.page_source
        return(html_page)
    except Exception as ex:
        print(ex)
    finally:
        driver.close()
        driver.quit()

# EAPTEKA.RU

In [17]:
def eap_apteka():
    
    city_id = ['petrozavodsk/', 'rostov-na-donu/', 'kazan/', 'smolensk/', 'krasnoyarsk/', 'chelyabinsk/', 
           'vladimir/', 'yaroslavl/', 'novokuznetsk/', 'krasnodar/', 'novorossiysk/', 'izhevsk/',
           'perm/', 'yoshkar-ola/', 'engels/', 'cheboksary/', 'tyumen/', 'astrakhan/', 'kaluga/', 'voronezh/',
           'belgorod/', 'kursk/', 'tambov/', 'ulan-udeh/','omsk/', 'tuapse/', 'syktyvkar/', 'samara/', 'abakan/',
           'tver/', 'stavropol/', 'magnitogorsk/', 'volgograd/', 'ryazan/', 'kirov/', 'lipetsk/', 'sochi/', 'ufa/',
           'pskov/', 'saransk/', 'ekb/', 'barnaul/', 'ulyanovsk/', 'briansk/', 'kostroma/', 'kemerovo/', 'ivanovo/',
           'makhachkala/', 'penza/', 'dzerzhinsk/', 'uhta/', 'saratov/', 'grozny/', 'tomsk/', 'pyatigorsk/', 'tula/',
           'vologda/', 'obninsk/', 'kurgan/']
    
    eap_data = {'site':[],
                'apteka':[],
                'agr':[],
                'nalichie':[],
                'region':[],
                'city':[],
                'address':[],
                'num_tab':[],
                'gramm':[],
                'price':[]}
    try:
        for i in city_id:
            city_link = 'https://www.eapteka.ru/' + i + 'search/?q=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE'

            main_page = get_html_page(city_link)
            soup = BeautifulSoup(main_page, 'html')
            city_name = soup.find('span', class_='select_title').get_text()
            pharmacy_id = soup.find_all('a', class_='cc-item--title')
            for i in pharmacy_id:
                try:
                    pharmacy_page = get_html_page('https://www.eapteka.ru' + i.get('href'))
                    soup = BeautifulSoup(pharmacy_page)
            
                    feature = soup.find('h1')
                    weight = re.findall(r'\d+', feature.get_text())[0]
                    amount = re.findall(r'\d+', feature.get_text())[-1]
                    html_price = soup.find('span', class_='offer-tools__price_num-strong')
                    price = html_price.get_text().strip().replace(' ','')
                    address_data = soup.find_all('td')
    
                    for i in address_data[::4]:
                        one_address = (i.get_text().replace('  ', '').replace('\n', ' '))
                        eap_data['address'].append(one_address.split(',', 1)[1].replace('г,', ''))
                        eap_data['city'].append(city_name)
                        eap_data['site'].append('https://www.eapteka.ru/')
                        eap_data['agr'].append('yes')
                        eap_data['nalichie'].append('')
                        eap_data['region'].append('')
                        eap_data['num_tab'].append(amount)
                        eap_data['gramm'].append(weight)
                        eap_data['price'].append(price)
                        eap_data['apteka'].append(re.findall(r'\w+', one_address)[0])
                except:
                    print('Что-то пошло не так в ' + city_link)
    except:
        print('Ссылка: ' + i + "не сработала")    
    df = pd.DataFrame.from_dict(eap_data)
    return(df)

In [18]:
start_time = time.time()
kek = eap_apteka()
kek
print("--- %s seconds ---" % (time.time() - start_time))

Что-то пошло не так в https://www.eapteka.ru/ekb/search/?q=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE
--- 3643.258195400238 seconds ---


In [19]:
kek

,site,apteka,agr,nalichie,region,city,address,num_tab,gramm,price
0,https://www.eapteka.ru/,ЕАПТЕКА,yes,,,Петрозаводск,"Октябрьский р-н, Зайцева ул, дом № 67, корпус а",28,50,2194
1,https://www.eapteka.ru/,ЕАПТЕКА,yes,,,Петрозаводск,"Октябрьский р-н, Зайцева ул, дом № 67, корпус а",28,200,2169
2,https://www.eapteka.ru/,ЕАПТЕКА,yes,,,Петрозаводск,"Октябрьский р-н, Зайцева ул, дом № 67, корпус а",28,100,2193
3,https://www.eapteka.ru/,ЕАПТЕКА,yes,,,Ростов-на-Дону,"Тружеников ул, дом № 39/3, строение 1",28,50,2290
4,https://www.eapteka.ru/,ЕАПТЕКА,yes,,,Ростов-на-Дону,"Тружеников ул, дом № 39/3, строение 1",28,200,2271
...,...,...,...,...,...,...,...,...,...,...
163,https://www.eapteka.ru/,ЕАПТЕКА,yes,,,Обнинск,"Ленина пр-кт, дом № 208, помещение 12",28,200,2149
164,https://www.eapteka.ru/,ЕАПТЕКА,yes,,,Обнинск,"Ленина пр-кт, дом № 208, помещение 12",28,100,2215
165,https://www.eapteka.ru/,ЕАПТЕКА,yes,,,Курган,"Коли Мяготина ул, дом № 100",28,50,2194
166,https://www.eapteka.ru/,ЕАПТЕКА,yes,,,Курган,"Коли Мяготина ул, дом № 100",28,200,2184


In [20]:
kek.to_csv("eap.csv") 

# Здоров.ру

In [4]:
def zdorov_parser():
    
    zdorov_data = {'site':[],
                'apteka':[],
                'agr':[],
                'nalichie':[],
                'region':[],
                'city':[],
                'address':[],
                'num_tab':[],
                'gramm':[],
                'price':[]} 
    
    main_page = get_html_page('https://zdorov.ru/catalog/search?q=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE')
    soup = BeautifulSoup(main_page, 'html')
    pharmacy = soup.find_all('a', class_='CatalogGoodItemstyled__FullTitle-sc-1wfmxei-4 irjaaE')
    price = soup.find_all('span', class_='PriceBlockstyled__PriceNumberActualBold-sc-1qxl8i0-17 hkIqch')
    
    driver = webdriver.Firefox(executable_path = PATH, options = options)
    driver.get('https://zdorov.ru')
    #time.sleep(5)
    apteka_address = driver.find_elements_by_xpath("//a")
    apteka_address[1].click()
    #time.sleep(5)
    html_address = driver.page_source
    driver.close()
    driver.quit()
    
    soup = BeautifulSoup(html_address, 'html')
    address_list = soup.find_all('div', class_='StockItemDesciptionstyled__Address-l3tfwc-18 kyIIRl')
    
    gorod_metro = soup.find_all('div', class_='StockItemDesciptionstyled__StockTitleContainer-l3tfwc-5 rlZNU')
    
    # тут находим граммовку и количество таблеток
    gramm = []
    amount = []
    prices = []
    address = []
    city = []
    
    for item in pharmacy:
            a = item.get_text()
            c = re.findall(r'\d+', a)
            gramm.append(c[0])
            amount.append(c[1])
    
    # тут находим цены
    for item in price:
        a = re.findall(r'\d+', item.get_text())
        c = a[0] + a[1]
        prices.append(c)
    
    # тут находим адреса
    soup = BeautifulSoup(html_address, 'html')
    address_list = soup.find_all('div', class_='StockItemDesciptionstyled__Address-l3tfwc-18 kyIIRl')
    for item in address_list:
        address.append(item.get_text())
    
    # тут находим город
    gorod_metro = soup.find_all('div', class_='StockItemDesciptionstyled__StockTitleContainer-l3tfwc-5 rlZNU')
    for i in gorod_metro:
        a = re.findall(r'\w+',i.get_text())
        if a[0] == 'г':
            city.append(a[1])
        else:
            city.append('Москва')
    
    for iter in range(len(address_list)):
        for i in range(len(pharmacy)):
            zdorov_data['address'].append(address[iter])
            zdorov_data['city'].append(city[iter])
            zdorov_data['price'].append(prices[i])
            zdorov_data['gramm'].append(gramm[i])
            zdorov_data['num_tab'].append(amount[i])
            zdorov_data['region'].append('Москва и МО')
            zdorov_data['site'].append('https://zdorov.ru')
            zdorov_data['agr'].append('')
            zdorov_data['nalichie'].append('')
            zdorov_data['apteka'].append('Здоров.ру')
    
    df = pd.DataFrame.from_dict(zdorov_data)
    return(df)

In [95]:
start_time = time.time()
kek = zdorov_parser()
kek
print("--- %s seconds ---" % (time.time() - start_time))

--- 12.67558741569519 seconds ---


In [96]:
kek

,site,apteka,agr,nalichie,region,city,address,num_tab,gramm,price
0,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Мастеркова, д.3",28,100,2179
1,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Мастеркова, д.3",28,200,2179
2,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Мастеркова, д.3",28,50,2171
3,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Мастеркова, д.3",56,50,4371
4,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Академика Янгеля, д.2",28,100,2179
...,...,...,...,...,...,...,...,...,...,...
339,https://zdorov.ru,Здоров.ру,,,Москва и МО,Электросталь,"ул. Корешкова, 3",56,50,4371
340,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Советская, д. 9",28,100,2179
341,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Советская, д. 9",28,200,2179
342,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Советская, д. 9",28,50,2171


In [97]:
kek.to_csv('zdorov.csv')
kek.to_excel('zdorov.xlsx')

# Столичные аптеки

In [5]:
def stolichniy():
    
    zdorov_data = {'site':[],
                'apteka':[],
                'agr':[],
                'nalichie':[],
                'region':[],
                'city':[],
                'address':[],
                'num_tab':[],
                'gramm':[],
                'price':[]}     
   
    URL ='https://www.sapteki.ru'
    main_page = get_html_page('https://www.sapteki.ru/catalog/?q=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE&s=')
    soup = BeautifulSoup(main_page, 'html')
    
    uperio_links = soup.find_all('a', class_='ph-link_orange ph-search__card-link')
    
    for link in uperio_links:
        preparat = get_html_page((URL + link.get('href')))
        soup = BeautifulSoup(preparat, 'html')
        
        uperio_features = soup.find_all('div', class_='col-12 col-md-7 col-lg-9')
        gramm_and_amount = re.findall(r'\d+', uperio_features[0].get_text())
        gramm = gramm_and_amount[0]
        amount = gramm_and_amount[5]

        address_and_price = soup.find_all('td')
        features = [item.get_text().strip() for item in address_and_price]
        address = features[1::7]
        price = features[6::7]
        
        for iter in range(len(address)):    
            zdorov_data['address'].append(address[iter])
            zdorov_data['city'].append('Москва')
            zdorov_data['price'].append(re.findall(r'\d+', price[iter].replace(' ', ''))[0])
            zdorov_data['gramm'].append(gramm)
            zdorov_data['num_tab'].append(amount)
            zdorov_data['region'].append('Москва и МО')
            zdorov_data['site'].append(URL)
            zdorov_data['agr'].append('')
            zdorov_data['nalichie'].append('')
            zdorov_data['apteka'].append('Столичные аптеки')

    df = pd.DataFrame.from_dict(zdorov_data)
    return(df)

In [73]:
start_time = time.time()
kek = stolichniy()
kek
print("--- %s seconds ---" % (time.time() - start_time))

--- 44.10305333137512 seconds ---


In [109]:
kek

,site,apteka,agr,nalichie,region,city,address,num_tab,gramm,price
0,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Севастопольский пр-т., д. 42, корп. 1",28,50,2 497 руб.
1,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Авиаконструктора Миля, д. 8, корп.1",28,50,2 509 руб.
2,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Братиславская, д. 19, корп. 1",28,50,2 533 руб.
3,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Троицкая, д.10 стр.1",28,50,2 537 руб.
4,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Лефортовский вал, д. 7",28,50,2 537 руб.
5,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Мясницкий пр-д, д.4, стр. 1",28,50,2 561 руб.
6,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Перовская, д. 10, корп. 1",28,50,2 561 руб.
7,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Ленинский пр-т, д. 12",28,50,2 561 руб.
8,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Вернадского пр-т., д. 51, стр. 2",28,100,2 520 руб.
9,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Перовская, д. 10, корп. 1",28,100,2 521 руб.


# Аптека Невис

In [6]:
def nevis():
    
    nevis_data = {'site':[],
                'apteka':[],
                'agr':[],
                'nalichie':[],
                'region':[],
                'city':[],
                'address':[],
                'num_tab':[],
                'gramm':[],
                'price':[]} 
    
    URL = 'https://aptekanevis.ru/'
    
    main_page = requests.get('https://aptekanevis.ru/catalog/poisk-preparata.php?s=&q=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE')
    
    
    soup = BeautifulSoup(main_page.text, 'html')
    # Цена
    price = soup.find_all('div', class_='price')
    uperio_price = [re.findall(r'\d+', item.get_text().replace(' ', '').replace('\n', ''))[0] for item in price]
    # граммовка и количество таблеток
    uperio_features = soup.find_all('span', class_='main')
    gramm = [re.findall(r'\d+', item.get_text())[0] for item in uperio_features]
    amount = [re.findall(r'\d+', item.get_text())[1] for item in uperio_features]
    # Ссылки
    uperio_card = soup.find_all('a', class_='image')
    
    for iterator in range(len(uperio_card)):
        uperio_page = requests.get(URL + uperio_card[iterator].get('href'))
        
        soup = BeautifulSoup(uperio_page.text, 'html')
        uperio_address = soup.find_all('div', class_='address')
        # Получение города, адреса и области
        i = 0
        gorod_list = []
        address_list = []
        oblast_list = []
        for item in uperio_address:
            full_address_string = item.get_text().strip()
            if re.findall(r'\w+', full_address_string)[0] == 'Санкт': # питерские разборки
                split_address = full_address_string.split(' г ')
                oblast_list.append('Ленинградская обл')
                gorod_list.append('Санкт-Петербург')
                try:
                    address_list.append(split_address[1])
                except:
                    address_list.append('10-я В.О. линия дом № 5')
            else:
                split_address = full_address_string.split(re.findall(r'\w+', full_address_string)[2])
                split_city_and_address = split_address[1].split(' г ')
                try:
                    address = split_city_and_address[1]
                    gorod = re.findall(r'\w+', split_city_and_address[0])[2]
                    oblast = split_address[0]
            
                    address_list.append(address)
                    gorod_list.append(gorod)
                    oblast_list.append(oblast)
                except:
                    i += 1
                    pass
        
        for iter in range(len(address_list)):    
            nevis_data['address'].append(address_list[iter])
            nevis_data['city'].append(gorod_list[iter])
            nevis_data['price'].append(uperio_price[iterator])
            nevis_data['gramm'].append(gramm[iterator])
            nevis_data['num_tab'].append(amount[iterator])
            nevis_data['region'].append(oblast_list[iter])
            nevis_data['site'].append(URL)
            nevis_data['agr'].append('')
            nevis_data['nalichie'].append('')
            nevis_data['apteka'].append('Аптека Невис')
    df = pd.DataFrame.from_dict(nevis_data)
    return(df)   

In [533]:
start_time = time.time()
kek = nevis()
kek
print("--- %s seconds ---" % (time.time() - start_time))

--- 24.325783014297485 seconds ---


In [534]:
kek

,site,apteka,agr,nalichie,region,city,address,num_tab,gramm,price
0,https://aptekanevis.ru/,Аптека Невис,,,Ленинградская область,Санкт-Петербург,Парфёновская ул дом № 9 корпус 1,28,100,2336
1,https://aptekanevis.ru/,Аптека Невис,,,Ленинградская область,Санкт-Петербург,10-я В.О. линия дом № 5,28,100,2336
2,https://aptekanevis.ru/,Аптека Невис,,,Ленинградская область,Санкт-Петербург,Малый В.О. пр-кт дом № 30-32,28,100,2336
3,https://aptekanevis.ru/,Аптека Невис,,,Ленинградская область,Волосово,Восстания ул дом № 17,28,100,2336
4,https://aptekanevis.ru/,Аптека Невис,,,Ленинградская область,Волосово,Вингиссара пр-кт дом № 26,28,100,2336
...,...,...,...,...,...,...,...,...,...,...
106,https://aptekanevis.ru/,Аптека Невис,,,Ленинградская область,Ивангород,Кингисеппское ш дом № 14а,56,200,4697
107,https://aptekanevis.ru/,Аптека Невис,,,Ленинградская область,Санкт-Петербург,Седова ул дом № 85,56,200,4697
108,https://aptekanevis.ru/,Аптека Невис,,,Ленинградская область,Санкт-Петербург,Большой П.С. пр-кт дом № 43,56,200,4697
109,https://aptekanevis.ru/,Аптека Невис,,,Ленинградская область,Санкт-Петербург,Пушкин,56,200,4697


In [537]:
kek.to_csv('nevis.csv')
kek.to_csv('nevis.xlsx')

# Овита

In [7]:
def get_list_of_cities():
    driver = webdriver.Firefox(executable_path = PATH, options=options)
    driver.get('https://ovita.ru/search/?word=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE')
    first_city = driver.find_element_by_class_name('first-city')
    first_city.click()
    city = driver.page_source
    driver.close()
    driver.quit()
    soup = BeautifulSoup(city, 'html')
    city_names = []
    city_html = soup.find_all('el')
    for item in city_html:
        if item.get_text() != '':
            city_names.append(item.get_text())
    return(city_names)

In [8]:
def uperio_amount(soup):
    amount = []
    features = soup.find_all('div', class_='product-description-name')
    for item in features:
        one_feature = re.findall(r'\d+', item.get_text())
        if len(one_feature) == 2:
            amount.append(one_feature[1])
        else:
            amount.append(one_feature[-1])
    return(amount)

In [9]:
def uperio_gramm(soup):
    gramm = []
    features = soup.find_all('div', class_='product-description-name')
    for item in features:
        one_feature = re.findall(r'\d+', item.get_text())
        if len(one_feature) == 2:
            gramm.append(one_feature[0])
        else:
            gramm.append(one_feature[0])
    return(gramm)

In [10]:
def get_price_list(soup):
    html_price = soup.find_all('span', class_='rub')
    price_list = [item.get_text().replace(' ','') for item in html_price[1:]]
    return(price_list)

In [11]:
def get_address_list(soup):
    apteka_addresses = soup.find_all('div', class_='map-element-name')
    apteka_addresses_list = [item.get_text() for item in apteka_addresses]
    none_list = soup.find_all('div', class_='map-element none')
    none_address_list = []
    for address_string in none_list:
        soup3 = address_string
        none_address = soup3.find('div', class_='map-element-name')
        none_address_list.append(none_address.get_text())
    clear_list = list(set(apteka_addresses_list) - set(none_address_list))
    return(clear_list)

In [12]:
def get_nalichie_list(soup):
    html_nalichie = soup.find_all('div', class_='map-element-amount')
    nalichie_list = []
    for item in html_nalichie:
        try:
            nalichie_list.append(re.findall(r'\d+', item.get_text())[0])
        except:
            pass
    return(nalichie_list)

In [13]:
def ovita(): # функция работает неправильно, предполагается для всех городов
    
    URL = 'https://ovita.ru'
    ovita_data = {'site':[],
                'apteka':[],
                'agr':[],
                'nalichie':[],
                'region':[],
                'city':[],
                'address':[],
                'num_tab':[],
                'gramm':[],
                'price':[]} 
    
    city_name = get_list_of_cities()
    driver = webdriver.Firefox(executable_path = PATH, options=options)
    
    driver.get('https://ovita.ru/search/?word=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE')
    first_city = driver.find_element_by_class_name('first-city')
    first_city.click()
    change_city = driver.find_elements_by_xpath('//el')
    for iterator in range(len(city_name)-1):
        change_city[iterator].click()
        city = driver.page_source
        soup = BeautifulSoup(city, 'html')
        
        amount_list = uperio_amount()
        gramm_list = uperio_gramm()
        price_list = get_price_list()
        
        uperio_card = soup.find_all('div', class_='amount-in-pharmacy')
        for item in range(len(uperio_card)):
            if uperio_card[item].get_text() != '':
                driver.get('https://ovita.ru/search/?word=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE')
                button = driver.find_elements_by_class_name('amount-in-pharmacy')
                button[item].click()                
                address_html_page = driver.page_source
                driver.close()
                driver.quit()
                soup = BeautifulSoup(address_html_page, 'html')
                
                address_list = get_address_list()
                nalichie_list = get_nalichie_list()
                
                for i in range(len(nalichie_list)):
                    ovita_data['address'].append(address_list[i])
                    ovita_data['city'].append(city_name[iterator])
                    ovita_data['price'].append(price[item])
                    ovita_data['gramm'].append(gramm[item])
                    ovita_data['num_tab'].append(amount[item])
                    ovita_data['region'].append('')
                    ovita_data['site'].append(URL)
                    ovita_data['agr'].append('')
                    ovita_data['nalichie'].append(nalichie_list[i])
                    ovita_data['apteka'].append('Овита')
    df = pd.DataFrame.from_dict(ovita_data)
    return(df)   

In [14]:
def ovita_parser(): # только для Владивостока
    URL = 'https://ovita.ru'
    ovita_data = {'site':[],
                'apteka':[],
                'agr':[],
                'nalichie':[],
                'region':[],
                'city':[],
                'address':[],
                'num_tab':[],
                'gramm':[],
                'price':[]} 
    
    main_page = get_html_page('https://ovita.ru/search/?word=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE')
    soup1 = BeautifulSoup(main_page, 'html')
    uperio_card = soup1.find_all('div', class_='amount-in-pharmacy') # лекарства которые есть в наличии
    
    amount_list = uperio_amount(soup1)
    gramm_list = uperio_gramm(soup1)
    price_list = get_price_list(soup1)
    
    for item in range(len(uperio_card)):
        if uperio_card[item].get_text() != '':
            driver = webdriver.Firefox(executable_path = PATH, options=options)
            driver.get('https://ovita.ru/search/?word=%D1%8E%D0%BF%D0%B5%D1%80%D0%B8%D0%BE')
            button = driver.find_elements_by_class_name('amount-in-pharmacy')
            button[item].click()
            address_html_page = driver.page_source
            soup2 = BeautifulSoup(address_html_page, 'html')
            
            address_list = get_address_list(soup2)
            nalichie_list = get_nalichie_list(soup2)
            driver.close()
            driver.quit()
            for i in range(len(nalichie_list)):
                ovita_data['address'].append(address_list[i])
                ovita_data['city'].append('Владивосток')
                ovita_data['price'].append(price_list[item])
                ovita_data['gramm'].append(gramm_list[item])
                ovita_data['num_tab'].append(amount_list[item])
                ovita_data['region'].append('')
                ovita_data['site'].append(URL)
                ovita_data['agr'].append('')
                ovita_data['nalichie'].append(nalichie_list[i])
                ovita_data['apteka'].append('Овита')
    df = pd.DataFrame.from_dict(ovita_data)
    return(df)

In [72]:
start_time = time.time()
kek = ovita_parser()
kek
print("--- %s seconds ---" % (time.time() - start_time))

--- 46.9842369556427 seconds ---


In [73]:
kek

,site,apteka,agr,nalichie,region,city,address,num_tab,gramm,price
0,https://ovita.ru,Овита,,1,,Владивосток,"пр-т 100 лет Вл-ку, 68а",28,100,2359
1,https://ovita.ru,Овита,,1,,Владивосток,"ул. Ладыгина, 3",28,100,2359
2,https://ovita.ru,Овита,,1,,Владивосток,"ул. Некрасовская, 100",28,100,2359
3,https://ovita.ru,Овита,,1,,Владивосток,"ул. Русская, 64",28,100,2359
4,https://ovita.ru,Овита,,1,,Владивосток,"пр-т 100 лет Вл-ку, 150в",28,100,2359
5,https://ovita.ru,Овита,,1,,Владивосток,"ул. Зои Космодемьянской, 5",28,100,2359
6,https://ovita.ru,Овита,,1,,Владивосток,"ул. Фокина, 27",28,100,2359
7,https://ovita.ru,Овита,,1,,Владивосток,"ул. Некрасовская, 61",28,100,2359
8,https://ovita.ru,Овита,,3,,Владивосток,"Народный пр-т, 20",28,100,2359
9,https://ovita.ru,Овита,,1,,Владивосток,"ул. Сахалинская, 5",28,100,2359


In [74]:
kek.to_csv('ovita.csv')

# Аптека Семейная

In [15]:
def apteka_semeinaya():
    
    URL = 'https://apteka-omsk.ru/'
    
    semeinaya_data = {'site':[],
                'apteka':[],
                'agr':[],
                'nalichie':[],
                'region':[],
                'city':[],
                'address':[],
                'num_tab':[],
                'gramm':[],
                'price':[]}
    
    driver = webdriver.Firefox(executable_path = PATH, options = options)
    driver.get(URL)
    enter_window = driver.find_elements_by_xpath('//input')
    enter_window[0].send_keys('юперио')
    enter_window[0].send_keys(Keys.ENTER)
    time.sleep(10)
    main_page = driver.page_source
    driver.close()
    driver.quit()
    
    soup = BeautifulSoup(main_page, 'html')
    html_name = soup.find_all('p', class_='col-md-5 title')
    html_price = soup.find_all('p', class_='col-md-2 price')
    html_address = soup.find_all('p', class_='col-md-3 adres')
    
    gramm_with = [re.findall(r'\d+', i.get_text())[0] for i in html_name]
    gramm = []
    for i in gramm_with:
        if i == '102':
            gramm.append('100')
        elif i == '51':
            gramm.append('50')
        else:
            gramm.append(i)
            
    amount = [re.findall(r'\d+', i.get_text())[-1] for i in html_name]
    price = [re.findall(r'\d+', i.get_text())[0] for i in html_price]
    address = [i.get_text() for i in html_address]
    
    for i in range(len(price)):
        semeinaya_data['address'].append(address[i])
        semeinaya_data['city'].append('Омск')
        semeinaya_data['price'].append(price[i])
        semeinaya_data['gramm'].append(gramm[i])
        semeinaya_data['num_tab'].append(amount[i])
        semeinaya_data['region'].append('')
        semeinaya_data['site'].append(URL)
        semeinaya_data['agr'].append('')
        semeinaya_data['nalichie'].append('')
        semeinaya_data['apteka'].append('Аптека семейная')
        
    df = pd.DataFrame.from_dict(semeinaya_data)
    return(df)     

In [25]:
start_time = time.time()
kek = apteka_semeinaya()
kek
print("--- %s seconds ---" % (time.time() - start_time))

['25', '25', '50', '25', '50', '100', '50', '25', '25', '100', '100', '50', '50', '25', '25', '25', '25', '50', '25', '50', '100', '50', '25', '50', '50', '50', '25', '100', '25', '50', '25', '25', '50', '100', '50', '50', '25', '100', '50', '100', '50', '50', '100', '25', '25', '50', '100', '100', '50', '50', '50', '50', '25', '50', '25', '50', '25', '100', '50']
--- 21.367985486984253 seconds ---


In [26]:
kek

,site,apteka,agr,nalichie,region,city,address,num_tab,gramm,price
0,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"пр. Мира, 88",28,25,2141
1,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"б.Заречный, 2а\t",28,25,2233
2,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"б.Заречный, 2а\t",28,50,2235
3,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул. Лермонтова, 20",28,25,2235
4,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул. Лермонтова, 20",28,50,2244
5,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул. Лермонтова, 20",28,100,2245
6,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"6я Станционная,2/3",28,50,2249
7,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"6я Станционная,2/3",28,25,2249
8,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"пр. Мира, 100",28,25,2255
9,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул. Заозерная, 11а",28,100,2260


In [16]:
def get_all_apteka():
    one_pharmacy = zdorov_parser()
    two_pharmacies = one_pharmacy.append(stolichniy())
    three_pharmacies = two_pharmacies.append(ovita_parser())
    four_pharmacies = three_pharmacies.append(nevis())
    five_pharmacies = four_pharmacies.append(apteka_semeinaya())
    return(five_pharmacies)

In [62]:
start_time = time.time()
kek = get_all_apteka()
print("--- %s seconds ---" % (time.time() - start_time))

--- 156.32259368896484 seconds ---


In [63]:
kek

,site,apteka,agr,nalichie,region,city,address,num_tab,gramm,price
0,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Севастопольский пр-т., д. 42, корп. 1",28,50,2 497 руб.
1,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Авиаконструктора Миля, д. 8, корп.1",28,50,2 509 руб.
2,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Братиславская, д. 19, корп. 1",28,50,2 533 руб.
3,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Троицкая, д.10 стр.1",28,50,2 537 руб.
4,https://www.sapteki.ru,Столичные аптеки,,,Москва и МО,Москва,"Лефортовский вал, д. 7",28,50,2 537 руб.
...,...,...,...,...,...,...,...,...,...,...
54,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул.Серова,16а",28,25,2518
55,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул.Серова,16а",28,51,2524
56,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул. Ватутина, 3",56,25,4588
57,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"пр. Маркса, 10",56,102,4612


In [60]:
kek.to_csv('output.csv')

In [28]:
if __name__ == '__main__':
    start_time = time.time()
    
    one_pharmacy = zdorov_parser()
    two_pharmacies = one_pharmacy.append(stolichniy())
    three_pharmacies = two_pharmacies.append(ovita_parser())
    four_pharmacies = three_pharmacies.append(nevis())
    five_pharmacies = four_pharmacies.append(apteka_semeinaya())
    six_pharmacies = five_pharmacies.append(eap_apteka())
    six_pharmacies.to_csv('apteka.csv')
    
    print("--- %s seconds ---" % (time.time() - start_time))

--- 156.07091808319092 seconds ---


In [29]:
five_pharmacies

,site,apteka,agr,nalichie,region,city,address,num_tab,gramm,price
0,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Мастеркова, д.3",28,100,2177
1,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Мастеркова, д.3",28,200,2179
2,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Мастеркова, д.3",28,50,2171
3,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Мастеркова, д.3",56,50,4449
4,https://zdorov.ru,Здоров.ру,,,Москва и МО,Москва,"ул. Академика Янгеля, д.2",28,100,2177
...,...,...,...,...,...,...,...,...,...,...
54,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул.Серова,16а",28,25,2518
55,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул.Серова,16а",28,50,2524
56,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"ул. Ватутина, 3",56,25,4588
57,https://apteka-omsk.ru/,Аптека семейная,,,,Омск,"пр. Маркса, 10",56,100,4612


In [80]:
five_pharmacies.to_csv('output.csv')